In [ ]:
import datetime
import weave
# Weave package now defaults to eager mode, but lazy mode required for this example notebook for now.
weave.use_lazy_execution()
import random
import time
from weave.ecosystem import shawn
from weave.ecosystem import wandb

In [ ]:
model_a = shawn.eval.EvalModel("a")
weave.save(model_a, 'model-a:latest')
proc = shawn.eval.PredictionProcess("000", model_a, [])
sample_time = datetime.datetime(2022, 10, 8, 10, 26)
for i in range(500):
    proc.predictions.append(shawn.eval.Prediction(sample_time, random.gauss(4, 1), random.gauss(5, 2)))
    sample_time += datetime.timedelta(seconds=1)
weave.save(proc, name="proc-a-1:latest")

model_b = shawn.eval.EvalModel("b")
weave.save(model_b, 'model-b:latest')
proc = shawn.eval.PredictionProcess("000", model_b, [])
sample_time = datetime.datetime(2022, 10, 8, 10, 31)
for i in range(500):
    proc.predictions.append(shawn.eval.Prediction(sample_time, random.gauss(2, 4), random.gauss(4, 2)))
    sample_time += datetime.timedelta(seconds=1)
weave.save(proc, name="proc-b-1:latest")
pass

In [ ]:
models = weave.ops.objects(shawn.eval.EvalModel.WeaveType(), 'latest', int(time.time()))

In [ ]:
panel = weave.panels.Group(
    preferHorizontal=True,
    items={
        "sidebar": weave.panels.Group(
            style="width: 200px; padding: 16px;",
            items={
                "model": weave.panels.ObjectPicker(models, label='model'),
                "baseline": weave.panels.ObjectPicker(models, label='baseline')
            }
        ),
        "main": lambda sidebar: weave.panels.Group(
            vars={
                'model_preds': weave.ops.execute(sidebar.config.items['model'].config.choice).get().all_predictions(
                    datetime.datetime(2022, 1, 1), datetime.datetime(2023, 1, 1)
                ),
                'baseline_preds': weave.ops.execute(sidebar.config.items['baseline'].config.choice).get().all_predictions(
                    datetime.datetime(2022, 1, 1), datetime.datetime(2023, 1, 1)
                ),
            },
            items={
                "input_plots": lambda model_preds, baseline_preds: weave.panels.Group(
                    style="height: 300px;",
                    preferHorizontal=True,
                    equalSize=True,
                    vars={
                        'models': weave.ops.make_list(
                            l0=weave.ops.dict_(
                                name='model',
                                preds=model_preds
                            ),
                            l1=weave.ops.dict_(
                                name='baseline',
                                preds=baseline_preds
                            )
                        )
                    },
                    items={
                        "input_plot": lambda models: weave.panels.LabeledItem(
                            label="Input",
                            item=weave.panels.Plot(
                                models,
                                x=lambda m: m['preds'].map(lambda r: r.timestamp),
                                y=lambda m: m['preds'].map(lambda r: r.input),
                                label=lambda model: model['name'],
                                mark='line'
                            )
                        ),
                        'input_dist': lambda models: weave.panels.LabeledItem(
                            label="Input Distribution",
                            item=wandb.Distribution(
                                models,
                                value_fn=lambda model: model['preds'].input,
                                label_fn=lambda model: model['name'],
                                bin_size=0.2
                            )
                        )
                    }
                ),
                "output_plots": lambda model_preds, baseline_preds: weave.panels.Group(
                    style="height: 300px;",
                    preferHorizontal=True,
                    equalSize=True,
                    vars={
                        'models': weave.ops.make_list(
                            l0=weave.ops.dict_(
                                name='model',
                                preds=model_preds
                            ),
                            l1=weave.ops.dict_(
                                name='baseline',
                                preds=baseline_preds
                            )
                        )
                    },
                    items={
                        "output_plot": lambda models: weave.panels.LabeledItem(
                            label="Output",
                            item=weave.panels.Plot(
                                models,
                                x=lambda m: m['preds'].map(lambda r: r.timestamp),
                                y=lambda m: m['preds'].map(lambda r: r.output),
                                label=lambda model: model['name'],
                                mark='line'
                            )
                        ),
                        'output_dist': lambda models: weave.panels.LabeledItem(
                            label="Output Distribution",
                            item=wandb.Distribution(
                                models,
                                value_fn=lambda model: model['preds'].output,
                                label_fn=lambda model: model['name'],
                                bin_size=0.2
                            )
                        )
                    }
                ),
#                 "tables": lambda model_preds: weave.panels.Group(
#                     style="height: 300px;",
#                     preferHorizontal=True,
#                     equalSize=True,
#                     items={
#                         "predictions_table": weave.panels.LabeledItem(
#                             label="Predictions Table",
#                             item=weave.panels.Table(
#                                 model_preds,
#                                 columns=[
#                                     lambda pred: pred.timestamp,
#                                     lambda pred: pred.input,
#                                     lambda pred: pred.output
#                                 ])
#                         )
#                     }),
            }
        )
    })
panel